In [1]:
import sys
sys.path.append(r"D:\Labber\Users\jpan\seqpy\src")
from seqpy import *
import matplotlib.pyplot as plt
%matplotlib widget

## Register pulses to sequence

In [5]:
x, y, z = sweepables("x y z")
a = 0.2 * Gaussian(width=.1e-6, plateau=.1e-6).shift(x)
b = 0.3 * Drag(width=.1e-6)
c = 0.4 * Cosine(width=.2e-6, plateau=.2e-6)
c1 = Carrier(phases=180, frequencies=100e6)
c2 = Carrier(phases=z, frequencies=50e6)
c3 = Carrier(phases=(0,0), frequencies=(25e6,75e6))
s = Sequence(n_channels=2)
s.register(0, a, c1, channel=0)
s.register(y, b, c2, channel=0)
s.register(1e-6, c, c3, channel=1)
s.trigger_pos = 1.5e-6
s.subs(x, -.1e-6)
s.subs(y, .5e-6)
s.subs(z, 90)6
s.plot().show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
s.waveforms(samp_freq=2.4e9)[0].shape

(4672,)

In [3]:
s.dump("test.json")
s.load("test.json")
s.subs("x", -.1e-6)
s.subs("y", .5e-6)
s.subs("z", 90)
s.trigger_pos = 150e-6
s.plot().show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
import zhinst.qcodes as ziqc
awg = ziqc.HDAWG("hdawg", "dev8319", interface="1gbe", host="127.0.0.1")
qa = ziqc.UHFQA("uhfqa", "dev2571", interface="1gbe", host="127.0.0.1")

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\Vostok\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\Vostok\.qcodes\logs\211013-41956-qcodes.log
Successfully connected to data server at 127.0.0.1:8004 api version: 6
Successfully connected to device DEV8319 on interface 1GBE
Connected to: Zurich Instruments HDAWG (serial:dev8319, firmware:67241) in 4.02s
Successfully connected to data server at 127.0.0.1:8004 api version: 6
Successfully connected to device DEV2571 on interface 1GBE
Connected to: Zurich Instruments UHFQA (serial:dev2571, firmware:67225) in 5.58s


In [5]:
update_zhinst_awg(sequence=s, awg=awg, period=200e-6*config.retrieve("SAMPLING_FREQUENCY"), repetitions=-1)

2021-10-13 14:24:05,851 ¦ zhinst.toolkit.control.drivers.base.awg ¦ WARNING ¦ awg ¦ compile ¦ 206 ¦ update_zhinst_awg(sequence=s, awg=awg, period=200e-6*config.retrieve("SAMPLING_FREQUENCY"), repetitions=-1)
Please check the sequencer code for hdawg-awg-0:

   11   wave w_1_1 = placeholder(112, true);
   12   while(true){
   13   playWave(1, w_0_0, 2, w_1_0);
-> 14   playZero(148700);
   15   playWave(1, w_0_1, 2, w_1_1);

Compiler status string:
Compilation started
Detected 1 devices with a total of 1 AWG cores.
Compiling source string
Warning (line: 14): play length 148700 is not aligned to 16 samples and will be extended to 148704 samples

Uploading ELF file to device dev8319



Upload of 2 waveforms took 0.012966 s


## Multichannel testing

In [6]:
x, y, z = sweepables("x y z")
a = 0.2 * Gaussian(width=.1e-6, plateau=.1e-6).shift(x)
b = 0.3 * Drag(width=.1e-6)
c = 0.4 * Cosine(width=.2e-6, plateau=.2e-6)
c1 = Carrier(phases=180, frequencies=100e6)
c2 = Carrier(phases=z, frequencies=50e6)
c3 = Carrier(phases=(0,0), frequencies=(25e6,75e6))
s = Sequence(n_channels=3)
s.register(0, a, c1, channel=0)
s.register(y, b, c2, channel=1)
s.register(1e-6, c, c3, channel=2)
s.trigger_pos = 1.5e-6
s.subs(x, -.1e-6)
s.subs(y, .5e-6)
s.subs(z, 90)
s.plot().show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
update_zhinst_awg(sequence=s, awg=awg, period=200e-6*config.retrieve("SAMPLING_FREQUENCY"), repetitions=-1, samp_freq=2.4e9)
awg.awgs[0].run()

Upload of 1 waveforms took 0.024931 s
Upload of 1 waveforms took 0.071806 s


In [8]:
control_width = 1e-7
readout_width = 500e-9
x, y = sweepables("x y")
X = .5 * Gaussian(width=control_width)
Y = y * Gaussian(width=control_width, plateau=x).shift(x/2)
R = Rect(width=readout_width).shift(readout_width/2)
cx = Carrier(phases=0, frequencies=25e6)
cy = Carrier(phases=90, frequencies=25e6)
cr = Carrier(phases=0, frequencies=50e6)
seq = Sequence(n_channels=2)
seq.register(-control_width*4, X, cx, channel=0)
seq.register(0, Y, cy, channel=0)
seq.register(x + control_width*4, X, cx, channel=0)
seq.register(x + control_width*7, R, cr, channel=1)
seq.trigger_pos = x + control_width*6
seq.subs(x, 1e-6)
seq.subs(y, 0.8)
seq.plot().show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
a = 0.5 * Rect(width=1.e-6)
c = Carrier(phases=0, frequencies=50e6)
seq = Sequence()
seq.register(0, a, c)
seq.plot().show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
# change the sampling rate to qa and change it back afterwards
update_zhinst_qa(qa, seq, samp_freq=1.8e9)

Current length of queue: 1


2021-10-13 14:24:27,080 ¦ zhinst.toolkit.control.drivers.base.awg ¦ WARNING ¦ awg ¦ compile ¦ 206 ¦ update_zhinst_qa(qa, seq, samp_freq=1.8e9)
Please check the sequencer code for uhfqa-awg-0:

    9   wave w_1_0 = placeholder(1808, false);
   10   while(true){
   11   waitDigTrigger(1, 1);
-> 12   setTrigger(AWG_MONITOR_TRIGGER);
   13   setTrigger(0);

Compiler status string:
Compilation started
Detected 1 devices with a total of 1 AWG cores.
Compiling source string
Warning (line: 12): constant 'AWG_MONITOR_TRIGGER' is deprecated, please use 'startQA' function

Uploading ELF file to device dev2571



Upload of 1 waveforms took 0.027926 s
